In [1]:
# 5-2
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

In [2]:
# 5-3
# =========================================================
# 目的：
#  - 日本語BERT（Whole Word Masking 版）を「MLM（Masked Language Modeling）」タスク用の
#    モデルとしてロードし、MacBook を含む環境で最適なデバイス（MPS/CUDA/CPU）に載せる。
#
# 理論メモ：
#  - MLM（Masked Language Modeling）は、入力トークンの一部を [MASK] に置換し、
#    その復元を学習する自己教師ありタスクである。BERTの事前学習の主要素。
#  - Whole Word Masking（WWM）は「サブワード単位ではなく“語”単位でまとめてマスクする」
#    戦略。語彙断片ではなく語レベルの完形復元圧力を与えるため、語彙的一貫性の向上が期待される。
#  - 推論（補完）時は [MASK] の位置の語彙分布（logits）から上位候補を選ぶ。
#    学習時は [MASK] 位置以外のロスを無視（ignore_index=-100）するのが一般的（DataCollatorが自動対応可）。
#  - 日本語BERT（BertJapaneseTokenizer）は「形態素解析→WordPiece」の二段で分割する。
#    分割は辞書・語彙・バージョンに依存するため、再現性は“性質（不変量）”で管理する。
#
# 実務Tips：
#  - MacBook（Apple Silicon）では CUDA は使えないため、Metal(MPS) を優先する。
#  - 推論（補完）だけなら model.eval() ＋ torch.no_grad() でメモリ＆速度最適化。
#  - 学習する場合は DataCollatorForLanguageModeling を用い、mlm_probability を設定する。
#  - トークナイザとモデルは必ず同じモデル名（語彙）を使う（IDずれ防止）。
# =========================================================

import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM


# --- デバイス検出（Mac優先：MPS → CUDA → CPU） ---
def get_best_device() -> torch.device:
    # Apple Silicon + macOS（MPSバックエンド）
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")
    # 他環境に移植した場合に備えて CUDA も許容
    if torch.cuda.is_available():
        return torch.device("cuda")
    # どれも不可なら CPU
    return torch.device("cpu")


device = get_best_device()
print(f"[info] using device = {device}")

# --- モデル名（WWM版日本語BERT）：トークナイザとモデルはペアで統一 ---
model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"

# トークナイザ（形態素解析→WordPiece）。特殊トークン：[CLS]/[SEP]/[PAD]/[MASK]/[UNK]
# - `mask_token` と `mask_token_id` は MLM の置換に使用する。
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# MLM 用のヘッド付き BERT（語彙サイズ×隠れ次元の出力層を持つ）
# - 出力：logits 形状 [B, L, |Vocab|]。各位置の語彙分布を返す。
bert_mlm = BertForMaskedLM.from_pretrained(model_name)

# モデルを最適デバイスへ移動（Mac では .cuda() ではなく .to(device) を使う）
bert_mlm = bert_mlm.to(device)

# 推論モード（補完タスクなど）：dropout を停止
bert_mlm.eval()

# --- 参考：MLM 推論の最小例（必要ならコメント解除して確認） ---
# text = f"明日は{tokenizer.mask_token}だ。"
# enc = tokenizer(text, return_tensors='pt')
# enc = {k: v.to(device) for k, v in enc.items()}
# with torch.no_grad():
#     logits = bert_mlm(**enc).logits  # [1, L, V]
# mask_index = (enc["input_ids"][0] == tokenizer.mask_token_id).nonzero(as_tuple=True)[0].item()
# topk = torch.topk(logits[0, mask_index], k=5).indices.tolist()
# candidates = [tokenizer.decode([i]) for i in topk]
# print("Top-5 candidates at [MASK]:", candidates)

# --- 参考：学習時のスケッチ ---
# from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments
# data_collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer,
#     mlm=True,
#     mlm_probability=0.15  # 15% を [MASK]（WWM版モデルの事前学習と整合的に設定）
# )
# # Trainer(...) を用いて学習。PAD位置は自動でロスから除外される（ignore_index=-100）。

[info] using device = mps


Some weights of the model checkpoint at tohoku-nlp/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [3]:
# 5-4
# =========================================================
# 目的：
#  - 入力文に含まれる [MASK] を含めて、日本語BERT用トークナイザでサブワード列へ分割し、挙動を確認する。
#
# 理論メモ（重要ポイント）：
#  - MLM（Masked Language Modeling）では、[MASK] 位置の語彙分布を予測する（学習時は [MASK] 以外の損失を無視）。
#  - `BertJapaneseTokenizer` は「形態素解析 → WordPiece」の二段で分割する。分割境界は辞書・語彙・バージョンに依存。
#  - `tokenize()` は **特殊トークン [CLS]/[SEP] を付けない**“素のサブワード列”を返す。
#  - `[MASK]` は **特殊トークンとして保存**される想定（= 文字列 "[MASK]" がそのままトークン列に現れる）。
#    ※ 安全のため、リテラル "[MASK]" 直書きより `tokenizer.mask_token` を使うのが堅牢。
#  - `encode()` / `tokenizer(..., return_tensors='pt')` は **ID列**を返し、既定で [CLS]/[SEP] を付与して下流モデルに直結できる。
# =========================================================

# 保険：tokenizer 未定義なら初期化（WWM版日本語BERTとペアで統一）
try:
    tokenizer
except NameError:
    from transformers import BertJapaneseTokenizer

    model_name = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

# --- 入力文：安全のため tokenizer.mask_token を用いて [MASK] を埋め込む ---
text = f"今日は{tokenizer.mask_token}へ行く。"

# --- サブワード列へ分割（特殊トークン [CLS]/[SEP] は付かない）---
tokens = tokenizer.tokenize(text)
print("# tokens:", tokens)

# --- ID への写像（[MASK] が mask_token_id に対応することを確認）---
ids = tokenizer.convert_tokens_to_ids(tokens)
print("# ids:", ids)

mask_tok = tokenizer.mask_token  # 例："[MASK]"
mask_id = tokenizer.mask_token_id  # 例：mask の語彙ID
mask_pos_tok = tokens.index(mask_tok) if mask_tok in tokens else None
mask_pos_id = ids.index(mask_id) if mask_id in ids else None
print(
    "# mask token:",
    mask_tok,
    "mask id:",
    mask_id,
    "pos(tok):",
    mask_pos_tok,
    "pos(id):",
    mask_pos_id,
)

# --- 参考：特殊トークン付与あり（モデル入力に近い形；[CLS] と [SEP] を含む）---
enc = tokenizer(text, return_tensors="pt", add_special_tokens=True)
print("# input_ids:", enc["input_ids"].tolist())
print("# attention_mask:", enc["attention_mask"].tolist())
print("# tokens (with specials):", tokenizer.convert_ids_to_tokens(enc["input_ids"][0]))

# --- 復元表示（可逆ではない場合がある点に注意：空白や正規化、未知語の影響）---
print(
    "# decode (skip specials):",
    tokenizer.decode(enc["input_ids"][0], skip_special_tokens=True),
)

# tokens: ['今日', 'は', '[MASK]', 'へ', '行く', '。']
# ids: [3246, 9, 4, 118, 3488, 8]
# mask token: [MASK] mask id: 4 pos(tok): 2 pos(id): 2
# input_ids: [[2, 3246, 9, 4, 118, 3488, 8, 3]]
# attention_mask: [[1, 1, 1, 1, 1, 1, 1, 1]]
# tokens (with specials): ['[CLS]', '今日', 'は', '[MASK]', 'へ', '行く', '。', '[SEP]']
# decode (skip specials): 今日 は へ 行く 。


In [4]:
# 5-5
# =========================================================
# 目的：
#  - [MASK] を含む文 `text` を符号化し、MLM（BertForMaskedLM）へ入力して
#    各トークン位置の語彙分布（logits）を得る。
#
# 理論メモ：
#  - MLM（Masked Language Modeling）では [MASK] 位置での語彙分布 p(token | context) を予測する。
#    モデル出力 `logits` は形状 [B, L, |V|]（バッチ×系列長×語彙サイズ）であり、
#    「分類スコア」は“各トークン位置における語彙分類のスコア”を意味する。
#  - `tokenizer.encode(text, return_tensors='pt')` は **ID列テンソル** を返す（既定で [CLS]/[SEP] 付与）。
#    研究・実務では `tokenizer(text, return_tensors='pt', padding=..., truncation=...)` の使用が推奨だが、
#    単文・短文であれば `encode` でも問題ない。
#  - attention_mask を省略した場合、PAD を含まない単文では実害は基本的にない。
#    ただしバッチやパディングを伴う運用では attention_mask を明示するのが安全。
#  - デバイス：MacBook(Apple Silicon) では CUDA は使えない。Metal(MPS) を優先し、
#    それも不可なら CPU を用いる。モデルとテンソルは**常に同じ device** に揃える。
# =========================================================

import torch


# --- デバイス検出（Mac優先：MPS → CUDA → CPU） ---
def get_best_device() -> torch.device:
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        return torch.device("mps")  # Apple GPU (Metal)
    if torch.cuda.is_available():
        return torch.device("cuda")  # 他環境への持ち出し用の保険
    return torch.device("cpu")


device = get_best_device()
print(f"[info] using device = {device}")

# 文章を符号化し、GPU（またはMPS/CPU）に配置する。
# - encode は既定で special tokens（[CLS]/[SEP]）を付与する。
# - return_tensors='pt' により torch.Tensor（形状 [B=1, L]）で取得。
input_ids = tokenizer.encode(
    text, return_tensors="pt"
)  # 例：tensor([[CLS, ..., MASK, ..., SEP]])
# 元コード： input_ids = input_ids.cuda()
# Mac では .cuda() は不可。device に合わせて移動する：
input_ids = input_ids.to(device)

# 念のためモデル側も同じデバイスへ（5-3でto(device)済みなら実質ノーオペ）
bert_mlm = bert_mlm.to(device)
bert_mlm.eval()  # 推論のみ。学習なら train() に切替

# BERTに入力し、語彙スコア（logits）を得る。
# ※ 元コメントの「系列長を揃える必要がない」は“単文かつ短文で PAD を使わない”場合に限って概ね正しい。
#    バッチや長文運用では padding / attention_mask を明示すること。
with torch.no_grad():  # 推論では計算グラフを作らない：省メモリ・高速化
    output = bert_mlm(
        input_ids=input_ids
    )  # 他に attention_mask 等を省略（単文短文の前提）
    scores = output.logits  # 形状 [B, L, |V|]：各位置の語彙ロジット

# --- 参考（任意）：[MASK] 位置の Top-k 候補を取得する手順（コメントアウト） ---
# mask_id = tokenizer.mask_token_id
# mask_pos = (input_ids[0] == mask_id).nonzero(as_tuple=True)[0].item()  # [MASK] の位置（L次元）
# topk = torch.topk(scores[0, mask_pos], k=5).indices.tolist()
# candidates = [tokenizer.decode([i]) for i in topk]  # 語彙ID → 文字列
# print("Top-5 candidates at [MASK]:", candidates)

# --- 参考（理論補足）---
# - Self-Attention：softmax(QK^T / √d_k) V。系列長 L に対して計算量 O(L^2 H)。
#   長文・バッチ運用では L を適切に管理（truncation/stride/動的パディング）すること。
# - PAD を含む場合は attention_mask=0 の位置が softmax 前に -∞ 相当で抑制され、注意が向かない（勾配も基本流れない）。

[info] using device = mps


In [5]:
# 5-6
# =========================================================
# 目的：
#  - MLM の出力 `scores`（形状 [B, L, |V|]）から [MASK] 位置の最尤トークン（argmax）を選び，
#    元の文の [MASK] をそのトークンで置換して，人間可読な文に復元する。
#
# 理論メモ：
#  - `scores[0, pos]` は語彙分布（ロジット）。argmax は MAP 推定（最尤語彙）に対応する。
#  - [MASK] の語彙IDを **固定値で仮定しない**（モデルによって異なる）。必ず `tokenizer.mask_token_id` を用いる。
#  - サブワード（WordPiece）の場合，`convert_ids_to_tokens` は '##' 接頭辞が付くことがある。
#    文字列復元としては `tokenizer.decode([id])` の方が自然なことが多い（空白処理や正規化を担う）。
#  - 複数 [MASK] がある場合：ここでは「独立同時置換」（各位置で argmax）を行う。
#    逐次的に再エンコードして条件付きで更新する「反復置換」は別戦略（より厳密だが計算増）。
# =========================================================

import torch

# --- 1) [MASK] の語彙IDを取得（モデル依存；固定値を使わない） ---
mask_id = (
    tokenizer.mask_token_id
)  # 例：東北大日本語BERTでは 4 のことが多いが、必ず動的に取得する
mask_tok = tokenizer.mask_token  # 例："[MASK]"

# --- 2) 入力ID列から [MASK] の位置を列挙（複数対応） ---
# input_ids: 形状 [B=1, L] を想定（5-5の encode に一致）
mask_positions = (input_ids[0] == mask_id).nonzero(as_tuple=True)[0].tolist()
if len(mask_positions) == 0:
    raise ValueError(
        "入力系列に [MASK] が見つかりません。text に tokenizer.mask_token を含めてください。"
    )

# --- 3) 各 [MASK] 位置で最尤トークンIDを取得（argmax over vocab） ---
# scores: 形状 [B=1, L, |V|] を想定（5-5の出力 logits）
pred_token_ids = []
for pos in mask_positions:
    # scores[0, pos] : [|V|] ベクトル → 最尤語彙ID
    best_id = scores[0, pos].argmax(-1).item()
    pred_token_ids.append(best_id)

# --- 4) ID → 文字列へ変換（decode を優先） ---
# decode は空白や '##' の扱いをよしなに調整してくれる（単一IDでも有用）
pred_tokens = [
    tokenizer.decode([tid], skip_special_tokens=True).strip() for tid in pred_token_ids
]

# フォールバック（必要時）：WordPieceの『##』を除去して素片文字列だけを得る
pred_tokens_fallback = [
    tokenizer.convert_ids_to_tokens(tid).lstrip("##") for tid in pred_token_ids
]

# decode 側で万が一空文字になった箇所はフォールバックを使う
pred_tokens = [
    pt if pt != "" else pf for pt, pf in zip(pred_tokens, pred_tokens_fallback)
]

# --- 5) 元のテキスト中の [MASK] を順に置換（複数 [MASK] に対応） ---
# str.replace は全置換になるため、分割→挿入→連結で「順番に」置換する
parts = text.split(mask_tok)  # [seg0, seg1, ..., segN]  （N = マスク数）
filled_segments = []
for i, seg in enumerate(parts[:-1]):
    filled_segments.append(seg)
    # i 番目の [MASK] に対する候補を挿入
    filled_segments.append(pred_tokens[i] if i < len(pred_tokens) else "")
filled_segments.append(parts[-1])

text_filled = "".join(filled_segments)
print(text_filled)

# --- 備考 ---
# - 「独立同時置換」では各 [MASK] を互いに独立に推定するため，相互依存（共起制約）を捉えにくい。
#   文脈整合性を高めたい場合は，1つずつ置換→再エンコード→再推論の反復法を検討する。
# - サブワードが選ばれた場合でも decode により自然な表記になりやすいが，
#   場合によっては不自然な連結になる可能性がある（語境界／表記揺れのため）。

今日は、へ行く。
